# Modelo Logistic Regression


transformação e modelo

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [120]:
# Carregamento dos dados
df = pd.read_csv("C:/Users/moura/Projetos/KDD_TeamOne/data/pre_processamento_3_train.csv")
df1 = pd.read_csv("C:/Users/moura/Projetos/KDD_TeamOne/data/pre_processamento_3_test.csv")

df1['DS_STATUS_ITEM'] = 0  # Exemplo com valores incrementais

df['DS_STATUS_ITEM_encoded'] = df['DS_STATUS_ITEM'].apply(lambda x: 0 if x == 'AUTORIZADO' else 1
)
df


,DS_TIPO_GUIA,IDADE,DS_TIPO_PREST_SOLICITANTE,DS_CBO,DS_CBO_ENCONDED,DS_INDICACAO_ACIDENTE,DS_CARATER_ATENDIMENTO,QT_DIA_SOLICITADO,DS_TIPO_ATENDIMENTO,CD_CID,DS_TIPO_ITEM,QT_SOLICITADA,DS_STATUS_ITEM,DS_STATUS_ITEM_encoded
0,GUIA DE SOLICITACAO SP/SADT,54,HOSPITAL,MEDICO ORTOPEDISTA E TRAUMATOLOGISTA,51,NAO ACIDENTE,URGENCIA/EMERGENCIA,0.0,PRONTO SOCORRO,R52,PROCEDIMENTOS,1.0,NEGADO,1
1,GUIA DE SOLICITACAO SP/SADT,57,PRESTADOR DE SERVICOS,MEDICO DERMATOLOGISTA,28,NAO ACIDENTE,ELETIVA,0.0,PEQUENA CIRURGIA,Z00,PROCEDIMENTOS,1.0,AUTORIZADO,0
2,GUIA DE SOLICITACAO INTERNACAO,45,HOSPITAL,MEDICO CLINICO,25,NAO ACIDENTE,URGENCIA/EMERGENCIA,3.0,INTERNACAO (SADT INTERNADO),B34,PROCEDIMENTOS,1.0,AUTORIZADO,0
3,GUIA DE SOLICITACAO SP/SADT,84,CLINICA,MEDICO CARDIOLOGISTA,16,NAO ACIDENTE,ELETIVA,0.0,EXAMES,I10,PROCEDIMENTOS,1.0,AUTORIZADO,0
4,GUIA DE SOLICITACAO SP/SADT,56,CLINICA,MEDICO ORTOPEDISTA E TRAUMATOLOGISTA,51,NAO ACIDENTE,ELETIVA,0.0,PEQUENA CIRURGIA,R52,MAT/MED,1.0,AUTORIZADO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413261,GUIA DE SOLICITACAO SP/SADT,61,MEDICO,MEDICO OFTALMOLOGISTA,49,NAO ACIDENTE,ELETIVA,0.0,EXAMES,H40,PROCEDIMENTOS,2.0,NEGADO,1
413262,GUIA DE SOLICITACAO SP/SADT,55,CLINICA,MEDICO OFTALMOLOGISTA,49,NAO ACIDENTE,ELETIVA,0.0,EXAMES,R52,PROCEDIMENTOS,1.0,AUTORIZADO,0
413263,GUIA DE SOLICITACAO DE PRORROGACAO DE INTERNACAO,59,HOSPITAL,MEDICO CIRURGI?O GERAL,21,NAO ACIDENTE,URGENCIA/EMERGENCIA,1.0,INTERNACAO (SADT INTERNADO),Z00,PROCEDIMENTOS,1.0,AUTORIZADO,0
413264,GUIA DE SOLICITACAO SP/SADT,53,CLINICA,MEDICO CLINICO,25,NAO ACIDENTE,ELETIVA,0.0,EXAMES,H40,PROCEDIMENTOS,2.0,AUTORIZADO,0


In [121]:
df.shape

(413266, 14)

In [122]:
df1.shape

(102410, 13)

In [123]:
# Empilhando os DataFrames verticalmente
df_empilhado = pd.concat([df, df1], ignore_index=True)

In [124]:
df_empilhado.shape

(515676, 14)

# treinamendo do modelo

In [111]:
#df_empilhado['DS_STATUS_ITEM_encoded'] = df_empilhado['DS_STATUS_ITEM'].apply(lambda x: 0 if x == 'A' else 1)

# Codificação de variáveis categóricas, exceto 'DS_STATUS_ITEM' que já foi convertido para binário
encoded_columns = pd.DataFrame()
for col in df_empilhado.select_dtypes(include='object').columns:
    # Ignorando a coluna DS_STATUS_ITEM original na codificação, pois já lidamos com ela
    if col != 'DS_STATUS_ITEM':
        le = LabelEncoder()
        encoded_columns[col + '_encoded'] = le.fit_transform(df_empilhado[col].astype(str))

In [131]:


# Criando um novo DataFrame para armazenar as colunas codificadas
encoded_columns = pd.DataFrame()

# Iterando sobre as colunas do tipo 'object', exceto 'DS_STATUS_ITEM'
for col in df_empilhado.select_dtypes(include='object').columns:
    if col != 'DS_STATUS_ITEM':  # Ignorando a coluna DS_STATUS_ITEM
        le = LabelEncoder()
        # Adicionando a coluna codificada ao DataFrame encoded_columns
        # Usamos .astype(str) para garantir que os dados sejam tratados como strings, caso não sejam originalmente
        encoded_columns[col + '_encoded'] = le.fit_transform(df_empilhado[col].astype(str))

#df_empilhado = pd.concat([df_empilhado, encoded_columns], axis=1)
# Padronização das dimensionalidades em X
scaler = StandardScaler()
encoded_columns.reset_index(drop=True, inplace=True)
X_standardized = pd.DataFrame(scaler.fit_transform(encoded_columns), columns=encoded_columns.columns)

encoded_columns['DS_STATUS_ITEM_encoded'] = df_empilhado['DS_STATUS_ITEM_encoded']

# Tamanho original dos DataFrames
tamanho_df = 413266  # Número de linhas em df
tamanho_df1 = 102410  # Número de linhas em df1

# Recuperando df do DataFrame empilhado
df = encoded_columns.iloc[:tamanho_df]

# Recuperando df1 do DataFrame empilhado
df1 = encoded_columns.iloc[tamanho_df:tamanho_df + tamanho_df1]

# Verificando os shapes para garantir que a divisão foi feita corretamente
print(f"Shape de df_separado: {df.shape}")
print(f"Shape de df1_separado: {df1.shape}")


Shape de df_separado: (413266, 9)
Shape de df1_separado: (102410, 9)


In [135]:
# Preparando os dados para modelagem
X = df.drop(['DS_STATUS_ITEM_encoded'], axis=1)
y = df['DS_STATUS_ITEM_encoded']

if X.empty or y.empty:
    raise ValueError("X ou y estão vazios. Verifique as operações de pré-processamento.")


In [136]:
y

0         1.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
413261    1.0
413262    0.0
413263    0.0
413264    0.0
413265    0.0
Name: DS_STATUS_ITEM_encoded, Length: 413266, dtype: float64

In [137]:
from sklearn.ensemble import RandomForestClassifier
# Divisão em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

# Instanciando o modelo
rf = RandomForestClassifier(random_state=42)

# Treinando o modelo
rf.fit(X_train, y_train)


# Previsões
y_pred = rf.predict(X_test)

# Avaliação
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7346286931062017
              precision    recall  f1-score   support

         0.0       0.75      0.91      0.83     57088
         1.0       0.63      0.34      0.44     25566

    accuracy                           0.73     82654
   macro avg       0.69      0.62      0.63     82654
weighted avg       0.72      0.73      0.71     82654



In [68]:
# Divisão em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

# Instanciando e treinando o modelo de Regressão Logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

LogisticRegression()

In [5]:
# Validação cruzada
kf = KFold(n_splits=5)
scores = cross_val_score(logistic_model, X, y, cv=kf)

print('*****************************************')
print(f"ESCORES DE VALIDAÇÃO CRUZADA: {scores}")
print(f"MÉDIA DOS ESCORES: {np.mean(scores)}")
print('*****************************************')

NameError: name 'logistic_model' is not defined

In [138]:
# Avaliação do modelo nos dados de teste
y_pred = rf.predict(X_test)
print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
print(f"Relatório de Classificação:\n{classification_report(y_test, y_pred)}")
print('*****************************************')
print(f"Matriz de Confusão:\n{confusion_matrix(y_test, y_pred)}")
print('*****************************************')

Acurácia: 0.7346286931062017
Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.75      0.91      0.83     57088
         1.0       0.63      0.34      0.44     25566

    accuracy                           0.73     82654
   macro avg       0.69      0.62      0.63     82654
weighted avg       0.72      0.73      0.71     82654

*****************************************
Matriz de Confusão:
[[52091  4997]
 [16937  8629]]
*****************************************


# Fazendo o dataset de submissão

In [140]:
df1

,DS_TIPO_GUIA_encoded,DS_TIPO_PREST_SOLICITANTE_encoded,DS_CBO_encoded,DS_INDICACAO_ACIDENTE_encoded,DS_CARATER_ATENDIMENTO_encoded,DS_TIPO_ATENDIMENTO_encoded,CD_CID_encoded,DS_TIPO_ITEM_encoded,DS_STATUS_ITEM_encoded
413266,2,0,49,0,0,2,503,1,NaN
413267,1,6,55,0,1,4,1413,1,NaN
413268,2,0,51,0,0,5,1413,0,NaN
413269,2,9,49,0,0,2,546,1,NaN
413270,2,8,50,0,0,2,124,1,NaN
...,...,...,...,...,...,...,...,...,...
515671,0,6,25,0,1,3,1413,1,NaN
515672,2,0,51,0,0,4,1637,0,NaN
515673,2,0,50,1,0,7,124,0,NaN
515674,1,6,33,0,1,4,620,1,NaN


In [167]:
# Preparando os dados para modelagem
X = df1.drop(['DS_STATUS_ITEM_encoded'], axis=1)
#y = df1.drop['DS_STATUS_ITEM_encoded']

if X.empty or y.empty:
    raise ValueError("X ou y estão vazios. Verifique as operações de pré-processamento.")

# Previsão no dataset de validação
y_pred_validation = rf.predict(X)

In [1]:
# Criando a coluna ID
ids = range(1, len(y_pred_validation) + 1)

# Criando o DataFrame com ID e previsões
results_df = pd.DataFrame({
    'ID': ids,
    'DS_STATUS_ITEM': int(y_pred_validation)
})


NameError: name 'y_pred_validation' is not defined

In [171]:
results_df

,ID,DS_STATUS_ITEM
0,1,0.0
1,2,0.0
2,3,1.0
3,4,0.0
4,5,0.0
...,...,...
102405,102406,0.0
102406,102407,0.0
102407,102408,0.0
102408,102409,0.0


In [144]:
results_df.to_csv('23_01_RandonForest_padrozicaojuntas.csv', index=False)
